In [ ]:
import sys
from pathlib import Path

from aiida import load_profile
from ase.io import read
from tb_hamiltonian.workflows import sweep_onsite_parameters

In [ ]:
sys.tracebacklimit = None

In [ ]:
_ = load_profile()

# Define parameters

In [ ]:
# Control parameters
debug = False
use_mpi = False

# Initial structure
structure_label = "BLG"
input_path = Path("examples/BLG/AB/rectangular")
initial_structure = read(input_path / "POSCAR", format="vasp")

# Supercell
nx = 1  # number of repetitions in x direction
ny = 1  # number of repetitions in y direction
nz = 1  # number of repetitions in z direction

## Distances
distances = [0.0, 1.425, 2.468, 2.850]

# Hamiltonian parameters
hopping_parameters = [0.0, -2.7, 0.0, -0.27]
interlayer_coupling = 0.33
nearest_neighbor = 1  # degree of nearest neighbors | don't use 0!

band_params = {
    "high_sym_points": {
        "Γ": (0.00000, 0.00000, 0.00000),
        "P": (0.00000, 0.33333, 0.00000),
        "X": (0.00000, 0.50000, 0.00000),
        "W": (0.50000, 0.50000, 0.00000),
        "Y": (0.50000, 0.00000, 0.00000),
    },
    "k_path": "Γ P X W Y Γ W",
    "points_per_segment": 100,
    "use_sparse_solver": False,
    "sparse_solver_params": {"k": 3, "sigma": 1e-8},
    "use_mpi": use_mpi,
    "fig_filename": "bands.png",
}

# Potential parameters
sweep_params = {
    "onsite_term": [],
    "potential_type": ["null", "kronig-penney", "sine"],
    "alpha": [[1.0, 0.3]],
    "potential_params": {
        "amplitude": [0.25, 0.5, 0.75],
        "width": [],
        "height": [],
    },
}

In [ ]:
computer = "localhost"

metadata = {
    "options": {
        # for more scheduler options, see https://aiida.readthedocs.io/projects/aiida-core/en/latest/topics/calculations/usage.html#options
        "prepend_text": "source $(conda info --base)/etc/profile.d/conda.sh && conda activate tbh",
    }
}

In [ ]:
wg = sweep_onsite_parameters(
    structure_label,
    initial_structure,
    [nx, ny, nz],
    distances,
    nearest_neighbor,
    hopping_parameters,
    interlayer_coupling,
    sweep_params,
    band_params,
    computer=computer,
    metadata=metadata,
)

In [ ]:
wg.submit()

In [ ]:
# ! verdi process status $wg.pk

In [ ]:
# from aiida import orm

# band_calculations = orm.QueryBuilder().append(
#     orm.WorkChainNode,
#     filters=orm.WorkChainNode.fields.pk == wg.pk,
#     tag="wg",
# ).append(
#     orm.CalcJobNode,
#     filters=orm.CalcJobNode.fields.label.like("get_band_structure%"),
#     with_incoming="wg",
# ).all()

# for calc, in band_calculations:
#     print(calc.label)
#     display(calc.outputs.fig.value)